# Código 02: Pontos

**Pipeline:** 


* Sistema de janela GLFW
    * Inicializa o sistema e o esconde
    * Cria janela e a torna principal
    * Captura eventos de teclado e mouse
* Shaders
    * Cria código fonte de Shader de vértice e fragmento em string
    * Requisita slot para a GPU para armazenar os códigos do shader, do vértice e do programa principal
    * Associa os códigos aos espaços recebidos
    * Compila os programas de vértice e de fragmento
    * Associa os programas compilados do vértice e do fragmento ao programa principal
    * Cria um executável do programa principal e define ele como o programa atual em uso
* Manipulação dos dados
    * Cria estrutura nparray com informações dos vértices
    * Solicita buffer de dados (espaço) da GPU para armazenar os vértices e o define como espaço onde será manipulados os dados
    * Preenche o buffer de dados com os dados dos vértices
* Link vértices e shader
    * Habilita atributo position
    * Faz o link do atributo position do Shader de vértice com os vértices
* Exibição: 
    * Exibe a janela
    * Loop principal: renderiza primitivas gráficas em cima das informações de pontos armzenadas nos vértices (printa os pontos e executa diretiva)

### Bibliotecas

In [3]:
#Bibliotecas
try:
    import glfw
    from OpenGL.GL import *
    import numpy as np
except ImportError:
    !pip install glfw
    !pip install pyopengl
    !pip install numpy
    import glfw
    from OpenGL.GL import *
    import numpy as np

### Inicializando janela

In [ ]:
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE)
window = glfw.create_window(720, 600, "Pontos", None, None)
glfw.make_context_current(window)

### Eventos do teclado e mouse

In [ ]:
def key_event(window, key, scancode, action, mods):
    print('[key event] key=',key)
    print('[key event] scancode=',scancode)
    print('[key event] action=',action)
    print('[key event] mods=',mods)
    print('-------')

glfw.set_key_callback(window, key_event)

def mouse_event(window, button, action, mods):
    print('[mouse event] button=',button)
    print('[mouse event] action=',action)
    print('[mouse event] mods=',mods)
    print('-------')

glfw.set_mouse_button_callback(window, mouse_event)

### GLSL

* "OpenGL Shading Language" (GLSF): linguagem de programação utilizada para escrever shaders (pequenos programas que são executados na placa de vídeo (GPU) para controlar aspectos específicos da renderização gráfica)
* Armazenamos o código em uma string (ou lendo arquivo de texto) o qual será compilado pelo código futuramente
* Usaremos GLSL antiga para ser compatível com os dispositivos

### GLSL para Vertex Shader

Código a seguir é responsável por processar cada vértice do objeto 2D. O código é útil para transformar as posições antes que sejam renderizadas na tela.

* `attribute vec2 position`: declara um atributo chamado `position` que contém um vetor bidimensional de coordenadas (vec2). Esse atributo será preenchido posteriormente com os dados de posição dos vértices. Em GLSL, atributo é uma variável especial usada para passar dados dos vértices dos objetos para shaders
* `void main(){}`: declara a função principal do Vertex Shader
* `gl_Position = vec4(position, 0.0, 1.0);`: declara a variável global `gl_Position`. Armazena a posição transformada de um vértice. As três primeiras cordenadas do vetor de quatro elementos é a dimensão (x,y,z). O último componente (componente homogêneo) é usado para normalização

Sobre `position` e `gl_Position`:
* `position`: variável de atributo que armazena temporariamente as coordenadas de posição de um único vértice durante o processo de renderização. Durante o pipeline de renderização, o OpenGL ou a API gráfica que você está usando processa cada vértice individualmente. Para cada vértice, as coordenadas de posição são lidas do buffer de vértices, passadas para o atributo position no shader e, em seguida, as transformações necessárias são aplicadas a esse vértice
* `gl_Position`: após todo o processamento, armazena a nova posição nesta variável. Quando o vértice for renderizado, ela vai ser utilizada por outro vértice

In [ ]:
vertex_code = """
        attribute vec2 position;
        void main(){
            gl_Position = vec4(position,0.0,1.0);
        }
        """

### GLSL para Fragment Shader

Fragment Shader é responsável por determinar a cor de cada fragmento (pixel) na tela após o processo de rasterização.
* `void(main){}`: declara a função principal do Fragment Shader
* `gl_FragColor`: variável global especial que representa a cor do fragmento atual.
* `vec4(0.0, 0.0, 0.0, 1.0)`: vetor de quatro componentes (RGBA) que representa uma cor preta totalmente opaca. Os três primeiros componentes (0.0, 0.0, 0.0) representam a intensidade de vermelho, verde e azul (preto não possui cor). O último componente (1.0) representa a opacidade total.

In [ ]:
fragment_code = """
        void main(){
            gl_FragColor = vec4(0.0, 0.0, 0.0, 1.0);
        }
        """

### Requisitando slot para a GPU para nossos programas Vertex e Fragment Shader

O SO armazena um espaço do código dos nossos programas automaticamente. Porém, como estamos criando um programa dentro de outro, precisamos solicitar um espaço manualmente.

* `glCreateProgram()`: cria um objeto programa vazio. Um objeto programa o qual objetos shaders podem ser anexados
* `glCreateShader()`: cria um objeto shader vazio. Um objeto shader é usado para manter o codigo fonte que define o shader

In [ ]:
program = glCreateProgram()
vertex = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)

### Associando os códigos fontes aos slots

* `glShaderSource()`: seta o código fonte no objeto shader. Qualquer código anteriormente armazenado no objeto Shader é sobrescrito

In [ ]:
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

### Compilando o Vertex Shader

* `glCompileShader()`: compila o código fonte que estiver armazenado no objeto shader. O status da compilação será armazenado no objeto shader.
* `glGetShaderiv()`: retorna um parametro, especificado via argumento, de um objeto shader
* `glGetShaderInfoLog()`: retorna informação de log de um objeto shader

In [ ]:
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")

### Compilando o Fragment Shader

In [ ]:
glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilação do Fragment Shader")

### Associando programas compilados ao programa principal

In [ ]:
#Anexa um objeto shader em um objeto programa
glAttachShader(program, vertex)
glAttachShader(program, fragment)

### Linkagem do programa

* Combinar os arquivos objeto de um programa em um único arquivo executável.

* `glLinkProgram()`: existindo algum objeto shader anexado ao objeto programa, a função irá criar um executável que será rodado no processador destinado a cada shader
* `glUseProgram()`: define o programa recém-vinculado como o programa de shader atual em uso

In [ ]:
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError("Link error")

glUseProgram(program)

### Sobre estruturas em python (resumo):

In [ ]:
import numpy as np

# Criação do array com 3 vértices
vertices = np.zeros(3, [("posic", np.float32, 2), ("texture_coord", np.float32, 2), ("color", np.float32, 4)])

# Preenchimento dos vértices com coordenadas e coordenadas de textura
vertices['posic'] = [
    ( 0.0, 0.0),  # vértice 0
    (0.5, 0.5),   # vértice 1
    (-0.5, 0.5)   # vértice 2
]

vertices['texture_coord'] = [
    (0.5, 0.0),   # textura para o vértice 0
    (1.0, 1.0),   # textura para o vértice 1
    (0.0, 1.0)    # textura para o vértice 2
]

vertices['color'] = [
    (1.0, 0.0, 0.0, 1.0),  # vermelho para o vértice 0
    (0.0, 1.0, 0.0, 1.0),  # verde para o vértice 1
    (0.0, 1.0, 0.0, 1.0)   # verde para o vértice 2
]

### Preparando dados para enviar para a CPU

As informações de vértices geralmente estão na CPU, mas queremos que elas vão para a GPU. Neste caso, usamos o código abaixo para criar três coordenaas para exibir três pontos em nosso programa.

In [ ]:
#Criando estrutura para receber 3 vértices com 2 coordenadas
vertices = np.zeros(3, [("posic", np.float32, 2)])

#Criando os 3 vértices
vertices['posic'] = [
                            ( 0.0, 0.0), # vertice 0
                            (+0.5,+0.5), # vertice 1
                            (-0.5,-0.5)  # vertice 2
                        ]

In [ ]:
#Gerando 500 pontos aleatórios
import random

QTD_PONTOS = 500
vertices = np.zeros(QTD_PONTOS, [("posic", np.float32, 2)])

pontos = []
for i in range(0, QTD_PONTOS):
    x = random.uniform(-1, 1)
    y = random.uniform(-1, 1)
    pontos.append((x,y))

vertices['posic'] = pontos

### Requisitando slot na GPU

Para enviar nossos dados da CPU para a GPU, precisamos solicitar slot

* `glGenBuffer(N)`: gera `N` identificadores para buffers de dados. Buffers de dados são usados para armazenar informações (vértices, texturas, etc.). Os identificadores gerados são usados para fazer referência aos buffers de dados na GPU. Depois de gerar o identificador,costuma-se preencher o buffer com os dados apropriados usando a função `glBufferData()`.
* `glBindBuffer(GL_ARRAY_BUFFER, buffer)`:  informa ao OpenGL que, a partir desse ponto, todas as operações que envolvem buffers de array (`GL_ARRAY_BUFFER`), como preencher com dados, manipular atributos de vértices, etc., serão realizadas no buffer especificado pelo identificador `buffer`.

In [ ]:
buffer = glGenBuffers(1)
glBindBuffer(GL_ARRAY_BUFFER, buffer)

### Enviar conteúdo da variável vértice

* `glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_DYNAMIC_DRAW)`: preenche o buffer na GPU com os dados `vertices`. `GL_ARRAY_BUFFER` indica que está trabalhando com um buffer de array. `vertices.nbytes` indica o tamanho em bytes dos dados está enviando para a GPU. `GL_DYNAMIC_DRAW`: indica que os dados serão frequentemente atualizados na GPU, indicando ao OpenGL que otimize o buffer para essa situação.

Estudos sobre os parâmetros da função acessar [glBufferData](https://registry.khronos.org/OpenGL-Refpages/gl4/html/glBufferData.xhtml]).

In [ ]:
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_DYNAMIC_DRAW)
#Qual a necessidade de usa DE NOVO glBindBuffer?
glBindBuffer(GL_ARRAY_BUFFER, buffer)

### Associando variáveis do programa GLSL (Vertex Shader) com nossos dados

Definimos byte inicial e offset dos dados
* `numpy.ndarray.strides[0]`: número de bytes que você precisa avançar na memória para acessar o próximo elemento no array. Outros elementos da tupla `numpy.ndarray.strides` só existem se o ndarray for irregular (ex: imagens)

Neste caso, como cada elemento de vertices é uma dupla de np.float32 (4 bytes), basta avançarmos 2x4 = 8bytes. Se fosse no exemplo hipotético dado no resumo de estruturas, seria 2x8 + 2x8 + 4x8 = 32 bytes

* `ctypes.c_void_p(N)`: cria valor especial que representa um "deslocamento N" em bytes dentro de um buffer. Esse valor será usado mais tarde junto com a função `glVertexAttribPointer()` para dizer ao OpenGL onde encontrar as informações corretas para cada atributo dos vértices.
* `ctypes`: módulo em Python que fornece facilidades para criar e manipular tipos de dados, funções, bibliotecas e ponteiros em C. Útil quando você precisa interagir com bibliotecas de linguagem C, como é o caso quando se trabalha com OpenGL

In [ ]:
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)
#print(vertices.strides[0])

Solicitamos à GPU a localização da variável "position" (que guarda coordenadas dos vértices criados) que definimos no Vertex Shader

* `glGetAttribLocation(program, "position")`: retorna o índice (localização) do atributo de vértice `position` dentro do programa de shader. A localização é um número inteiro que identifica a posição do atributo dentro do shader.
* `glEnableVertexAttribArray(loc)`: No contexto da renderização gráfica com OpenGL, os vértices de um objeto podem ter vários atributos, como posição, cor, coordenadas de textura, entre outros. Para que o OpenGL saiba como usar esses atributos durante a renderização, você precisa habilitar os atributos que deseja usar. Esta função diz que o atributo de vértice referente a essa localização seja considerado e usado durante o processo de renderização.

In [ ]:
loc = glGetAttribLocation(program, "position")
#Dúvida: vou ter que habilitar sempre que for adicionar informações sobre o ponto?
glEnableVertexAttribArray(loc)

Sabemos onde, na GPU, está o conteúdo que a variável position vai processar

 * `glVertexAttribPointer(loc, 2, GL_FLOAT, False, stride, offset)`: informa ao OpenGL como acessar e interpretar os dados dos atributos de vértices no buffer, permitindo que eles sejam corretamente passados para os shaders durante a renderização.
    * `loc`: índice do atributo de vértice que você está configurando
    * `2`: número de componentes para cada atributo de vértice. Neste caso, (x,y).
    * `GL_FLOAT`: tipo de dado dos componentes do atributo de vértice
    * `False`: indica se os valores devem ser normalizados
    * `stride`: espaçamento em bytes entre os dados de vértices consecutivos no buffer
    * `offset`: deslocamento em bytes do início do buffer onde começam os dados do atributo. Isso é usado para especificar onde exatamente os dados desse atributo estão armazenados dentro do buffer.

Mais detalhes em [glVertexAttribPointer()](https://registry.khronos.org/OpenGL-Refpages/gl4/html/glVertexAttribPointer.xhtml)

In [ ]:
glVertexAttribPointer(loc, 2, GL_FLOAT, False, stride, offset)

### Exibe a janela

In [ ]:
glfw.show_window(window)

### Loop principal

Novidade é glDrawArrays()
* `glDrawArrays(GL_POINTS, 0, 3)`: renderiza primitivas gráficas (como pontos, linhas ou triângulos) com base nos dados armazenados nos buffers de vértices. 
    * `GL_POINTS`: especifica o tipo de primitiva para renderizar
    * `0`: índice do primeiro vértice para renderizar
    * `3`: número de vértices que você deseja renderizar


Os pontos são processados primeiros e as primitivas logo depois, mas como o resultado é armazenado no buffer o qual será trocado na função `glfw.swap_buffers()`, o resultado final é exibido junto.

In [ ]:
while not glfw.window_should_close(window):
    glfw.poll_events()

    glClear(GL_COLOR_BUFFER_BIT)
    glClearColor(1.0,1.0,1.0,1.0)

    glDrawArrays(GL_POINTS, 0 , QTD_PONTOS)

    glfw.swap_buffers(window)

glfw.terminate()